In [40]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from google.cloud import storage
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.window import Window

In [41]:
spark = SparkSession.builder.appName('Nifty50').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '30g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','30g'), ("spark.driver.maxResultSize", "70g")])

gcs_client = storage.Client()

In [42]:
bucket_name = 'bigdata_nifty50'

In [43]:
bucket = gcs_client.bucket(bucket_name)

In [44]:
file_name = 'df_with_indicators.csv'

In [45]:
df = spark.read.csv('gs://{}//{}'.format(bucket_name, file_name), inferSchema=True)

In [46]:
new_columns = ['date',
 'close',
 'high',
 'low',
 'open',
 'volume',
 'sma5',
 'sma10',
 'sma15',
 'sma20',
 'ema5',
 'ema10',
 'ema15',
 'ema20',
 'upperband',
 'middleband',
 'lowerband',
 'HT_TRENDLINE',
 'KAMA10',
 'KAMA20',
 'KAMA30',
 'SAR',
 'TRIMA5',
 'TRIMA10',
 'TRIMA20',
 'ADX5',
 'ADX10',
 'ADX20',
 'APO',
 'CCI5',
 'CCI10',
 'CCI15',
 'macd510',
 'macd520',
 'macd1020',
 'macd1520',
 'macd1226',
 'MFI',
 'MOM10',
 'MOM15',
 'MOM20',
 'ROC5',
 'ROC10',
 'ROC20',
 'PPO',
 'RSI14',
 'RSI8',
 'slowk',
 'slowd',
 'fastk',
 'fastd',
 'fastksr',
 'fastdsr',
 'ULTOSC',
 'WILLR',
 'ATR',
 'Trange',
 'TYPPRICE',
 'HT_DCPERIOD',
 'BETA',
 'sector',
 'company',
 'Inflation Rate ',
 'Balance of Trade',
 'Bank Lending Rate',
 'Car Registrations',
 'Consumer Price Index',
 'Crude Oil Production',
 'Fiscal Expenditure',
 'Industrial Production',
 'Food Inflation',
 'Producer Prices',
 'Reverse Repo Rate',
 'Steel Production',
 'Tourist Arrivals',
 'Corporate Tax Rate ',
 'Export Prices',
 'GDP per Capita PPP',
 'GDP',
 'Gross National Product',
 'Import Prices',
 'Military Expenditure']     

In [47]:
from functools import reduce

old_columns = df.schema.names
df = reduce(lambda data, idx: data.withColumnRenamed(old_columns[idx], new_columns[idx]), range(len(old_columns)), df)

In [48]:
cols = df.columns
cols.remove('company')
cols.remove('sector')

exprs = {x: "avg" for x in cols}
exprs['sector'] = 'first'
date_df = df.groupby(df.company, to_date(df.date).alias('day')).agg(exprs)

In [49]:
w = Window.partitionBy('company').orderBy("day")

date_df = date_df.withColumn('diffOpenClose', col("avg(open)") - col("avg(close)"))
date_df = date_df.withColumn('diffHighLow', col("avg(high)") - col("avg(low)"))
date_df = date_df.withColumn('target', when(lag(col("avg(close)")).over(w) < col("avg(close)"), 1).otherwise(0))

In [51]:
indexer1 = StringIndexer(inputCol="company", outputCol="companyIdx").setHandleInvalid("skip")
indexer2 = StringIndexer(inputCol="first(sector)", outputCol="sectorIdx").setHandleInvalid("skip")

inputs = [indexer1.getOutputCol(), indexer2.getOutputCol()]

encoder = OneHotEncoder(inputCols=inputs,  \
                                 outputCols=["companyVec", "sectorVec"])

#run it through a pipeline
pipeline = Pipeline(stages=[indexer1, indexer2, encoder])
date_df1 = pipeline.fit(date_df).transform(date_df)

date_df1.show(5)

+--------+----------+----------------------+------------------+----------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------------+-----------------+---------------------+------------------+---------------------+---------------------------+------------------+--------------------+-------------------+-------------------+------------------+------------------+-----------------------+------------------+------------------+--------------------+------------------+--------------------+--------------------+------------------+-------------------+-----------------------+------------------+-------------------+-------------------------+------------------+------------------+-------------------+--------------------+-------------------+------------+-------------------+------------------+------------------+---------------

In [52]:
date_df1 = date_df1.withColumn('day', unix_timestamp(date_df1.day))

In [53]:
cols = date_df1.columns
for col in ['company', 'first(sector)', 'target']:
    cols.remove(col)

cols.remove('companyIdx')
cols.remove('sectorIdx')
    
cols

['day',
 'avg(Reverse Repo Rate)',
 'avg(RSI14)',
 'avg(Car Registrations)',
 'avg(CCI10)',
 'avg(TRIMA10)',
 'avg(SAR)',
 'avg(fastdsr)',
 'avg(ADX20)',
 'avg(WILLR)',
 'avg(sma10)',
 'avg(macd1226)',
 'avg(sma20)',
 'avg(ema15)',
 'avg(ema10)',
 'avg(KAMA10)',
 'avg(TRIMA20)',
 'avg(Tourist Arrivals)',
 'avg(fastksr)',
 'avg(Steel Production)',
 'avg(Gross National Product)',
 'avg(TRIMA5)',
 'avg(macd1020)',
 'avg(CCI5)',
 'avg(MOM15)',
 'avg(Export Prices)',
 'avg(TYPPRICE)',
 'avg(GDP per Capita PPP)',
 'avg(HT_DCPERIOD)',
 'avg(ema5)',
 'avg(open)',
 'avg(Producer Prices)',
 'avg(PPO)',
 'avg(ADX10)',
 'avg(CCI15)',
 'avg(Fiscal Expenditure)',
 'avg(Import Prices)',
 'avg(ATR)',
 'avg(Crude Oil Production)',
 'avg(upperband)',
 'avg(low)',
 'avg(BETA)',
 'avg(ROC20)',
 'avg(macd520)',
 'avg(date)',
 'avg(MOM10)',
 'avg(fastk)',
 'avg(MFI)',
 'avg(Corporate Tax Rate )',
 'avg(APO)',
 'avg(sma5)',
 'avg(KAMA20)',
 'avg(RSI8)',
 'avg(macd510)',
 'avg(lowerband)',
 'avg(Military Expe

In [54]:
assembler = VectorAssembler(inputCols=cols, outputCol="features")

In [55]:
date_df1 = assembler.transform(date_df1)

In [56]:
date_df1 = date_df1.withColumnRenamed('target', 'label')

In [57]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label', numTrees=10)

In [58]:
pipeline = Pipeline(stages=[rf])

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 15, 20]) \
    .build()

crossVal = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

In [ ]:
cvModel = crossVal.fit(date_df1)

In [68]:
cvModel.save('gs://{}//models//cvrfModel'.format(bucket_name))

In [67]:
predictions2 = cvModel.transform(date_df1)

In [69]:
e = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction")
print(e.evaluate(predictions2, {e.metricName: "areaUnderROC"}))
print(e.evaluate(predictions2, {e.metricName: "areaUnderPR"}))

0.7791020746007526


0.7547791071051762
